In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import pickle

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext autoreload
%autoreload 2
import opticalmaterials
import prisms

# Load glass catalog

In [ ]:
%%time
with open("compoundprisms/rii.pickle", "rb") as f:
    catalog = pickle.load(f)

# Singlet

In [ ]:
dispersion_target = np.deg2rad(1)  # deg
w = np.linspace(0.4, 0.7, 20)
n1_func = opticalmaterials.get_n(catalog[("SCHOTT-LaK", "N-LAK33A")])
n1 = n1_func(w)

In [ ]:
prisms.optimize_singlet(w, n1, dispersion_target)

In [ ]:
[c for c in catalog.keys() if "SCHOTT" in c[0]]

In [ ]:
deltaT_target = np.deg2rad(10)
w = np.linspace(0.4, 0.7, 20)
rows = []
for glass_name, glass in catalog.items():
    if "SCHOTT" not in glass_name[0]:
        continue
    try:
        n1_func = opticalmaterials.get_n(glass)
        n1 = n1_func(w)
        row = prisms.optimize_singlet(w, n1, deltaT_target)
    except:
        continue
    rows.append((*glass_name, *row))

In [ ]:
rows[2]

In [ ]:
singlet_df = pd.DataFrame.from_records(
    rows,
    columns=(
        "glass1",
        "glass2",
        "beta",
        "gamma",
        "alpha",
        "deltaC",
        "deltaM",
        "delta1",
        "deltaT",
        "nonlin",
        "NL",
        "SSR",
    ),
)
for col in ["beta", "gamma", "alpha", "deltaC", "deltaM", "delta1"]:
    singlet_df.loc[:, col] *= 180 / np.pi

In [ ]:
singlet_df.sort_values("SSR")

## Doublet

In [ ]:
glasses = [
    "N-PK51",
    "N-BK10",
    "N-BAF4",
    "N-KZFS11",
    "N-BAF52",
    "N-KZFS4",
    "N-LAK33A",
    "N-BALF5",
    "N-LASF31A",
    "N-BAF4",
    "LAFN7",
    "N-BASF2",
    "N-LASF44",
    "N-BAF10",
    "N-LAF34",
    "N-SSK5",
    "N-SF66",
    "N-LAK14",
]

glasses_long = [[gg for gg in catalog.keys() if gg[1] == g][0] for g in glasses]

In [ ]:
%%time
deltaT_target = np.deg2rad(20)
merit = "dev_linearity"
w = np.linspace(0.4, 0.7, 20)
rows = []
for glass_name1 in glasses_long:
    try:
        n1_func = opticalmaterials.get_n(catalog[glass_name1])
        n1 = n1_func(w)
    except:
        continue
    for glass_name2 in glasses_long:
        try:
            n2_func = opticalmaterials.get_n(catalog[glass_name2])
            n2 = n2_func(w)
            row = prisms.optimize_doublet(w, n1, n2, deltaT_target, merit=merit)
        except:
            continue
        rows.append((*glass_name1, *glass_name2, *row))

In [ ]:
doublet_df = pd.DataFrame.from_records(
    rows,
    columns=(
        "glass1_1",
        "glass1_2",
        "glass2_1",
        "glass2_2",
        "alpha1",
        "alpha2",
        "deltaC",
        "deltaT",
        "NL",
        "SSR",
        "deltaM",
        "K",
        "delta1",
        "delta2",
        "nonlin",
        "chromat",
    ),
)
for col in ["alpha1", "alpha2", "deltaC", "deltaT", "deltaM", "delta1", "delta2"]:
    doublet_df.loc[:, col] *= 180 / np.pi

In [ ]:
doublet_df.sort_values("SSR")

In [ ]:
doublet_df[["deltaT", "SSR", "K"]].hvplot.scatter("deltaT", "SSR", color="K")

In [ ]:
w2 = np.linspace(0.35, 1, 100)
k1 = opticalmaterials.get_k(catalog[("SCHOTT-LaSF", "N-LASF44")])
plt.plot(w2, opticalmaterials.transmittance(w2, k1(w2), 1e5))

# double-Amici

In [ ]:
%%time
deltaT_target = np.deg2rad(20)
merit = "dev_linearity"
w = np.linspace(0.4, 0.7, 20)
rows = []
for glass_name1 in glasses_long:
    try:
        n1_func = opticalmaterials.get_n(catalog[glass_name1])
        n1 = n1_func(w)
    except:
        continue
    for glass_name2 in glasses_long:
        try:
            n2_func = opticalmaterials.get_n(catalog[glass_name2])
            n2 = n2_func(w)
            row = prisms.optimize_doublet(w, n1, n2, deltaT_target, merit=merit)
        except:
            continue
        rows.append((*glass_name1, *glass_name2, *row))

# Triplet

# Janssen quintuplet